In [1]:
# Model design
import agentpy as ap

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
import random

In [2]:
# in starting points (13,0), the car should be able to also go to the another track
# in other words to (14,0), according to this grid this is the formost left lane

# in starting points (0,13), the car should be able to algo fo to the another track
# in other words (0,12) for example, accordint to this gris this is the foremost upway lane

# in starting points (12,24), the car should be able to also go to the another track
# in other words to (11,24), according to this grid this is the foremost right lane

#in starting points (24,14), the car should be able to also go to the another track
# in other words to (24,13), according to this grid this is the formost left lane

startingCoordinates = [(13,0),(0,11),(11,24),(24,13)]

movement1 = (0,1) # right
movement2 = (1,0) # down
movement3 = (0,-1) # left
movement4 = (-1,0) # up


In [3]:
class streetIntersection(ap.Model):

    def setup(self):

        #define the street limiter, that which will be black
        square = int((self.p.size-5)*2) #squares of size -5 as the resting number represents the street
        self.roadLimit_xy = ap.AgentList(self,square)
        self.roadLimit_x_y = ap.AgentList(self,square)
        self.roadLimitxy = ap.AgentList(self,square)
        self.roadLimitx_y = ap.AgentList(self,square)

        #define car agents
        self.carsl = ap.AgentList(self, self.p.n_cars)
        self.carsu = ap.AgentList(self, self.p.n_cars)
        self.carsr = ap.AgentList(self, self.p.n_cars)
        self.carsd = ap.AgentList(self, self.p.n_cars)

        self.singleCar = ap.AgentList(self,1)



        #Create street grid
        self.street = ap.Grid(self,[self.p.size]*2,track_empty=True)
        #add each delimiter
        carspos1 = [startingCoordinates[0] for i in range(len(self.carsl))]
        carspos2 = [startingCoordinates[1] for i in range(len(self.carsl))]
        carspos3 = [startingCoordinates[2] for i in range(len(self.carsl))]
        carspos4 = [startingCoordinates[3] for i in range(len(self.carsl))]

        self.street.add_agents(self.carsl,positions=carspos1)
        self.street.add_agents(self.carsu,positions=carspos2)
        self.street.add_agents(self.carsr,positions=carspos3)
        self.street.add_agents(self.carsd,positions=carspos4)


        #CONDITIONS -> 0:delimiters 1:cars 2:inactive
        self.roadLimit_xy.condition = 0
        
        self.carsl.condition = 2
        self.carsu.condition = 1
        self.carsr.condition = 3
        self.carsd.condition = 4


    
    def step(self):
        #we need to select and separate cars in different conditions as they move differently

        # left cars
        moving_carsl = self.carsl.select(self.carsl.condition == 2)
        # up cars
        moving_carsu = self.carsu.select(self.carsu.condition == 1)
        # right cars
        moving_carsr = self.carsr.select(self.carsr.condition == 3)
        # down cars
        moving_carsd = self.carsd.select(self.carsd.condition == 4)

        # left cars
        
        for i, car in enumerate(moving_carsl):

            currentPos = self.street.positions[car] # get current position
            movementL = movement1[1] # movement in the y axis of the car
            frontPos = (currentPos[0], currentPos[1] + movementL + 1) # position of the possible car in front

            if (frontPos[1] < 25): # condition to check if we don't iterate out of the grid

                print("current: " + str(currentPos))
                print("front: " + str(frontPos))
                print("casilla front: " + str(self.street.agents[frontPos].condition))
                print("casilla carros: " + str(self.street.agents[currentPos].condition))
                print("starting up: " + str(self.street.agents[startingCoordinates[1]].condition))

                if (self.street.agents[frontPos].condition == 2 and len(self.street.agents[currentPos].condition) < 2): # if there is a car in front and the current cell doesn't have multiple cars
                    print("non empty cell")
                    print("___")
                    continue
                else:
                    print("empty cell")
                    print("___")
                    self.street.move_by(car, movement1)

                self.street.move_by(car, movement1)
        
        # up cars
        for i, car2 in enumerate(moving_carsu):
                
            currentPos = self.street.positions[car2]
            movementU = movement2[0]
            frontPos = (currentPos[0] + movementU + 1, currentPos[1])

            if (frontPos[0] < 25):

                print("current: " + str(currentPos))
                print("front: " + str(frontPos))
                print("casilla front: " + str(self.street.agents[frontPos].condition))
                print("casilla carros: " + str(self.street.agents[currentPos].condition))
                print("starting up: " + str(self.street.agents[startingCoordinates[1]].condition))

                if (self.street.agents[frontPos].condition == 1 and len(self.street.agents[currentPos].condition) < 2):
                    print("non empty cell")
                    print("___")
                    continue
                else:
                    print("empty cell")
                    print("___")
                    self.street.move_by(car2, movement2)

                self.street.move_by(car2, movement2)

        # right cars
        for i, car3 in enumerate(moving_carsr):
                
            currentPos = self.street.positions[car3]
            movementR = movement3[1]
            frontPos = (currentPos[0], currentPos[1] + movementR - 1)

            if (frontPos[1] > -1):

                print("current: " + str(currentPos))
                print("front: " + str(frontPos))
                print("casilla front: " + str(self.street.agents[frontPos].condition))
                print("casilla carros: " + str(self.street.agents[currentPos].condition))
                print("starting up: " + str(self.street.agents[startingCoordinates[1]].condition))

                if (self.street.agents[frontPos].condition == 3 and len(self.street.agents[currentPos].condition) < 2):
                    print("non empty cell")
                    print("___")
                    continue
                else:
                    print("empty cell")
                    print("___")
                    self.street.move_by(car3, movement3)

                self.street.move_by(car3, movement3)
        
        # down cars
        for i, car4 in enumerate(moving_carsd):

            currentPos = self.street.positions[car4]
            movementD = movement4[0]
            frontPos = (currentPos[0] + movementD - 1, currentPos[1])

            if (currentPos[0] == 0):

                continue

            if (frontPos[0] > -1):
                    
                    print("current D: " + str(currentPos))
                    print("front: " + str(frontPos))
                    print("casilla front: " + str(self.street.agents[frontPos].condition))
                    print("casilla carros: " + str(self.street.agents[currentPos].condition))
                    print("starting up: " + str(self.street.agents[startingCoordinates[1]].condition))
    
                    if (self.street.agents[frontPos].condition == 4 and len(self.street.agents[currentPos].condition) < 2):
                        print("non empty cell")
                        print("___")
                        continue
                    else:
                        print("empty cell")
                        print("___")
                        self.street.move_by(car4, movement4)
    
                    self.street.move_by(car4, movement4)
            


In [4]:
parameters = {
    'size': 25,
    'steps': 15,
    'n_cars': 10,
}

In [5]:
# Create single-run animation with custom colors

def animation_plot(model, ax):
    attr_grid = model.street.attr_grid('condition')
    color_dict = {0:'#000000', 1:'#0000FF',2:'#FFFFFF', 3:'#FF5733', 4:'#33FFC4', None: '#d5e5d5'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"TESTING STREET\n")

fig, ax = plt.subplots()
model = streetIntersection(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=12))

current: (13, 0)
front: (13, 2)
casilla front: []
casilla carros: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
starting up: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
empty cell
___
current: (13, 0)
front: (13, 2)
casilla front: [2]
casilla carros: [2, 2, 2, 2, 2, 2, 2, 2, 2]
starting up: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
empty cell
___
current: (13, 0)
front: (13, 2)
casilla front: [2, 2]
casilla carros: [2, 2, 2, 2, 2, 2, 2, 2]
starting up: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
empty cell
___
current: (13, 0)
front: (13, 2)
casilla front: [2, 2, 2]
casilla carros: [2, 2, 2, 2, 2, 2, 2]
starting up: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
empty cell
___
current: (13, 0)
front: (13, 2)
casilla front: [2, 2, 2, 2]
casilla carros: [2, 2, 2, 2, 2, 2]
starting up: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
empty cell
___
current: (13, 0)
front: (13, 2)
casilla front: [2, 2, 2, 2, 2]
casilla carros: [2, 2, 2, 2, 2]
starting up: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
empty cell
___
current: (13, 0)
front: (13, 2)
casilla front: [2, 2, 2, 2, 2, 2]
casilla 